<a href="https://colab.research.google.com/github/bharathbolla/EVA4/blob/S4/Bharath__EVA4_Session_4_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Sequential(nn.Conv2d(1,16,3,bias=False),           #conv1  1 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),
                                 nn.BatchNorm2d(16),                     #o/p:26
                                 nn.Dropout2d(0.05))
        
        self.conv2=nn.Sequential(nn.Conv2d(16,32,3,bias=False),          #conv2  16 input , 32 0utput and 3x3 kernel
                                 nn.ReLU(),
                                 nn.BatchNorm2d(32),                     #o/p:24
                                 nn.Dropout2d(0.05))
        
        # used 1x1 kernel 
        self.conv3=nn.Sequential(nn.Conv2d(32,8,1,bias=False))           #conv3  32 input , 8 0utput and 1x1 kernel 
                                 
                                                                         #o/p:24
        
        self.maxpool=nn.MaxPool2d(2,2)                                   # maxpooling 2x2 kernel and stride 2
        
                                                                         #o/p:12

        self.conv4=nn.Sequential(nn.Conv2d(8,16,3,bias=False),           #conv4  8 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),
                                 nn.BatchNorm2d(16),                     #o/p:10
                                 nn.Dropout2d(0.05))
        
        self.conv5=nn.Sequential(nn.Conv2d(16,16,3,bias=False),          #conv5  16 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),
                                 nn.BatchNorm2d(16),                     #o/p:8
                                 nn.Dropout2d(0.05))
        
       # self.maxpool=nn.MaxPool2d(2,2)
        
        self.conv6=nn.Sequential(nn.Conv2d(16,16,3,padding=1,bias=False),  #conv6  16 input , 16 0utput and 3x3 kernel
                                 nn.ReLU(),
                                 nn.BatchNorm2d(16),                      #o/p:8
                                 nn.Dropout2d(0.05))
        
        self.conv7=nn.Sequential(nn.Conv2d(16,10,3,bias=False))            #conv7  16 input , 10 0utput and 3x3 kernel
        
                                                                          #o/p:6 
                                  
        
        self.GAP=nn.AvgPool2d(6)                         # Gobal Average Pooling 

                                                         #o/p:1


    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.maxpool(x)
        x=self.conv4(x)
        x=self.conv5(x)
        x=self.conv6(x)
        x=self.conv7(x)
        x=self.GAP(x)
        x=x.view(-1,10)
        return F.log_softmax(x)





In [0]:
!pip install torchsummary

In [3]:

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
         Dropout2d-8           [-1, 32, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             256
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,152
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
        Dropout2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0726, Accuracy: 9793/10000 (97.93%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0463, Accuracy: 9864/10000 (98.64%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0204, Accuracy: 9924/10000 (99.24%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0201, Accuracy: 9932/10000 (99.32%)



loss=0.015213902108371258 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.59it/s]



Test set: Average loss: 0.0190, Accuracy: 9940/10000 (99.40%)



## Hi
### Hi